In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from pandas.api.types import CategoricalDtype
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory
from sklearn import model_selection, ensemble, metrics, linear_model
import matplotlib.pyplot as plt
from sklearn.preprocessing import *
import os
base_dir = '../input'
print(os.listdir(base_dir))


# Any results you write to the current directory are saved as output.https://www.kaggle.com/wanermiranda/linear-regression-ml-tp1?scriptVersionId=5240484

In [ ]:
df_diamonds = pd.read_csv('%s/diamonds.csv'%(base_dir), index_col='Unnamed: 0')
df_diamonds.head(10)

## Numeric Features 
* Carat: weight of the diamond
* depth: depth %  The height of a diamond, measured from the culet to the table, divided by its average girdle diameter
* table: table % The width of the diamond's table expressed as a percentage of its average diameter
* price: the price of the diamond
* xlength: mm
* ywidth: mm
* zdepth: mm

In [ ]:
df_diamonds.describe()

## cut 
Describe cut quality of the diamond. Quality in increasing order Fair, Good, Very Good, Premium, Ideal

In [ ]:
cuts_ordered = ['Fair',
                'Good',
                'Very Good',
                'Premium',
                'Ideal']
df_diamonds['cut'] = df_diamonds['cut'].astype(CategoricalDtype(cuts_ordered, ordered=True))
print(df_diamonds['cut'].unique())
df_diamonds['cut'].describe()

## color
mColor of the diamond, with D being the best and J the worst

In [ ]:
colors_ordered = [  'J',
                    'I',
                    'H',
                    'G',
                    'F',
                    'E',
                    'D']
df_diamonds['color'] = df_diamonds['color'].astype(CategoricalDtype(colors_ordered, ordered=True))
print(df_diamonds['color'].unique())
df_diamonds['color'].describe()

## clarity
How obvious inclusions are within the diamond:(in order from best to worst, FL = flawless, I3= level 3 inclusions) FL,IF, VVS1, VVS2, VS1, VS2, SI1, SI2, I1, I2, I3

In [ ]:
clarity_codes = {'I3',
'I2',
'I1',
'SI2',
'SI1',
'VS2',
'VS1',
'VVS2',
'VVS1',
'IF',
'FL'}
df_diamonds['clarity'] = df_diamonds['clarity'].astype(CategoricalDtype(clarity_codes, ordered=True))
print(df_diamonds['clarity'].unique())
df_diamonds['clarity'].describe()

## Cleaning the Data
There are some zero dimensions for the diamonds, since that must be noise or mistype, we are cleaning it.


In [ ]:
df_diamonds = df_diamonds.drop(df_diamonds.loc[df_diamonds.x <= 0].index)
df_diamonds = df_diamonds.drop(df_diamonds.loc[df_diamonds.y <= 0].index)
df_diamonds = df_diamonds.drop(df_diamonds.loc[df_diamonds.z <= 0].index)

## Handcraft features
Since the measures for the diamond follow a 3d shape, we are considering here some handcraft features. 
Volume for the diamond = reflecting its size and weight. 
Ratio between the X, Y and Z.


In [ ]:
df_diamonds['volume'] = df_diamonds['x'] * df_diamonds['y'] * df_diamonds['z']
df_diamonds['ratio'] = df_diamonds['x'] / df_diamonds['y']
df_diamonds['ratio'] = df_diamonds['x'] / df_diamonds['z']
df_diamonds.head(10)

In [ ]:
train, test_reserved = model_selection.train_test_split(df_diamonds, test_size=0.2, random_state=42)
test_reserved.to_csv('test.csv')
train.to_csv('train.csv')
# test_reserved = pd.read_csv('test.csv', index_col='Unnamed: 0')
# train = pd.read_csv('train.csv', index_col='Unnamed: 0')
df_diamonds = train

# Distribution Overview
The prices seems to follow a power law curve, as show bellow in the graph. 

In [ ]:
df_diamonds['price'].hist(bins=100)

In [ ]:
df_diamonds['price'].describe()

## SGD Regression For Fun


In [ ]:
cat_columns = df_diamonds.select_dtypes(['category']).columns.values
df_diamonds[cat_columns] = df_diamonds[cat_columns].apply(lambda x: x.cat.codes)



## Normalizing the Data
Using the robust scaller **to not only use the mean normalization**, but also to be less vulnerable to outliers.

In [ ]:
X  = df_diamonds.copy()
y = X.pop('price')
scaler = RobustScaler()
scaler.fit(X)
X = scaler.transform(X)


## Regression
Since there is no negative values in the prices we are using here the log(price) to maintain this domain during the regression train. 
We are also using a 5 cross fold validation to do the grid search. 

A validation set was extracted from the data as a simulation for the test set.


In [ ]:
from sklearn.model_selection import *

X_train, X_val, y_train, y_val = model_selection.train_test_split(X, y, test_size=0.1, random_state=42)


In [ ]:

    
params = {
    'learning_rate':['invscaling', 'optimal'],
    'eta0': [0.1, 0.05, 0.01], 
    'max_iter':[20000]
}

scoring = {
    'NEG_MSE': 'neg_mean_squared_error',
    'NEG_MAE': 'neg_mean_absolute_error',
    'VARIANCE': 'r2'
}

best_params = {'eta0': [0.01], 'learning_rate': ['invscaling'], 'max_iter': [10000]}
best_params_ = {'eta0': 0.01, 'learning_rate': 'invscaling', 'max_iter': 10000}

## Removing the penalty because it could lead to throubles
## when implementing the regression
regr = linear_model.SGDRegressor(**best_params_, penalty=None, verbose=True) 
# regr_ = GridSearchCV(regr_, best_params, cv=2,
#                        scoring=scoring, refit='VARIANCE',
#                     n_jobs=-1,
#                    verbose=True
#                    )

#regr = GridSearchCV(linear_model.SGDRegressor(),params,cv=3, scoring=scoring, refit='VARIANCE', n_jobs=-1, verbose=True)

regr.fit(X_train, np.log(y_train))
# results = regr.cv_results_


In [ ]:
# regr.best_score_

In [ ]:
# regr.best_params_

In [ ]:
# def GridSearch_table_plot(grid_clf, param_name,score,
#                           num_results=15,                          
#                           negative=True,
#                           graph=True,
#                           display_all_params=True):

#     '''Display grid search results

#     Arguments
#     ---------

#     grid_clf           the estimator resulting from a grid search
#                        for example: grid_clf = GridSearchCV( ...

#     param_name         a string with the name of the parameter being tested

#     num_results        an integer indicating the number of results to display
#                        Default: 15

#     negative           boolean: should the sign of the score be reversed?
#                        scoring = 'neg_log_loss', for instance
#                        Default: True

#     graph              boolean: should a graph be produced?
#                        non-numeric parameters (True/False, None) don't graph well
#                        Default: True

#     display_all_params boolean: should we print out all of the parameters, not just the ones searched for?
#                        Default: True

#     Usage
#     -----

#     GridSearch_table_plot(grid_clf, "min_samples_leaf")

#                           '''
#     from matplotlib      import pyplot as plt
#     from IPython.display import display
#     import pandas as pd

#     clf = grid_clf.best_estimator_
#     clf_params = grid_clf.best_params_
#     if negative:
#         clf_score = -grid_clf.best_score_
#     else:
#         clf_score = grid_clf.best_score_
#     clf_stdev = grid_clf.cv_results_['std_test_'+score][grid_clf.best_index_]
#     cv_results = grid_clf.cv_results_

#     print("best parameters: {}".format(clf_params))
#     print("best score:      {:0.5f} (+/-{:0.5f})".format(clf_score, clf_stdev))
#     if display_all_params:
#         import pprint
#         pprint.pprint(clf.get_params())

#     # pick out the best results
#     # =========================
#     scores_df = pd.DataFrame(cv_results).sort_values(by='rank_test_'+score)

#     best_row = scores_df.iloc[0, :]
#     if negative:
#         best_mean = -best_row['mean_test_'+score]
#     else:
#         best_mean = best_row['mean_test_'+score]
#     best_stdev = best_row['std_test_'+score]
#     best_param = best_row['param_' + param_name]

#     # display the top 'num_results' results
#     # =====================================
#     display(pd.DataFrame(cv_results) \
#             .sort_values(by='rank_test_'+score).head(num_results))

#     # plot the results
#     # ================
#     scores_df = scores_df.sort_values(by='param_' + param_name)

#     if negative:
#         means = -scores_df['mean_test_'+score]
#     else:
#         means = scores_df['mean_test_'+score]
#     stds = scores_df['std_test_'+score]
#     params = scores_df['param_' + param_name]

    
# GridSearch_table_plot(regr, param_name="eta0", score='VARIANCE', negative=False)        

In [ ]:
# regr.best_estimator_

In [ ]:
y_val.describe()

In [ ]:
y_pred = np.exp(regr.predict(X_val))
pd.Series(y_pred).describe()

In [ ]:
print("MSE: %.3f" % metrics.mean_squared_error(y_val, y_pred))
print("MAE: %.3f" % metrics.mean_absolute_error(y_val, y_pred))
print('R2: %.3f' % metrics.r2_score(y_val, y_pred))

plt.hist(y_val, bins=100, color='blue', linewidth=3)
plt.show()
plt.hist(y_pred, bins=100, color='red', linewidth=3)
plt.show()

In [ ]:
df = pd.DataFrame({'real': y_val, 'pred': y_pred})
ax = df.sort_values('real').plot.scatter('real', 'pred', figsize=(5, 5))
_ = ax.plot([y.min(), y.max()], [y.min(), y.max()], 'k--', lw=3)

In [ ]:
def oneHotEncoding(features,columnName):
	currentCol = features.columns.get_loc(columnName)
	uniqueFeatures = features[columnName].unique()
	print(uniqueFeatures)
	for f in range(len(uniqueFeatures)):
		features.insert(loc=currentCol+f,column=columnName+str(f),value=0)
		features[columnName+str(f)][features[columnName]==uniqueFeatures[f]] = 1
		
	features.pop(columnName)

def dummieCoding(features,columnName,orderedFeature):
	c = 0
	for f in range(len(orderedFeature)):
		features[columnName][features[columnName]==orderedFeature[f]] = 2**c
		c = c + 1

In [ ]:
def SMSE(parameters,features,target,j):
	npFeatures = features.values
	h = (np.sum(np.multiply(parameters,npFeatures))-target) * npFeatures[j]
	return h


def SGD(alpha, iterations, features, target):
	features.insert(0,"theta0",1)
	shape = features.shape
	nsamples = shape[0]
	print("Number of samples: "+str(nsamples))
	nparams = shape[1]
	print("Number of parameters: "+str(nparams))

	parameters = np.zeros(nparams)
	new_parameters = np.zeros(nparams)

	error = 1
	epsilon = 0.0001
	it = 0
	i = 0

	while ((error > epsilon) and (it < iterations) and (i < nsamples)):
		for j in range(nparams):
			new_parameters[j] = parameters[j] - alpha *             SMSE(parameters,features.ix[i],target.ix[i],j)		
		it += 1
		i += 1
		error = math.sqrt(np.sum(np.power(np.subtract(new_parameters,parameters),2)))
		print(parameters)
		print(new_parameters)
		np.copyto(parameters,new_parameters)
		print("Epoch: "+str(it))
		print("Sample: "+str(i))
		print("Error: "+str(error))
		print("\n\n")

	features.pop("theta0")

	return parameters

In [ ]:
theta = np.array([1, 0, 1], dtype=np.double)
theta_temp = np.array([0, 0, 0], dtype=np.double)
y = np.array([5.,10.], dtype=np.double)
X = np.array([[0.,1., 2.],[0.,2., 3.]], dtype=np.double)
print (X)
alpha = .01
max_iter = 50

In [ ]:
def hyphotesis(theta, X):
    return np.sum(theta.T * X, axis=1)
    
def MSE_theta(theta, X, y, alpha,j, h0, error):                
        S = np.sum(np.matmul(error, X[:,j]))                
        result = theta[j] - (alpha * (1. / len(y)) * S)        
        return result

for i in range(max_iter):
    h0 = hyphotesis(theta, X)
    error = (h0 - y)
    for j in range(X.shape[1]):
        theta_temp[j] = MSE_theta(theta, X, y, alpha, j, h0, error)    
        
    theta = theta_temp.copy()
    print (theta)    

hyphotesis(theta, X)

In [ ]:
import math

import math


def SGD_(alpha, max_iter, X, y):
    
    # Creating theta0 
    X = np.insert(X, values=1, obj=0, axis=1)
    
    shape = X.shape
    nsamples = shape[0]
    print("Number of samples: "+str(nsamples))
    theta0 = np.zeros(nsamples)
    nparams = shape[1]
    print("Number of parameters: "+str(nparams))


    theta = np.random.uniform(size=nparams)
    theta_temp = np.ones(nparams)

    error = 1
    epsilon = 0.001
    it = 0
    i = 0   
    power_t = 0.25
    t=1.0
    
    while ((error > epsilon) and (it < max_iter)):
        h0 = hyphotesis(theta, X)
        eta = alpha / pow(t, power_t)
        error = (h0 - y)
        for j in range(nparams):
            theta_temp[j] = MSE_theta(theta, X, y, eta, j, h0, error)                
        it += 1
        i += 1
        y_pred = hyphotesis(theta_temp, X)
#         print (y,hyphotesis(theta_temp, X))
        error =  ((y - y_pred) ** 2).mean() / 2 
#         print(theta)
#         print(theta_temp)

        theta = theta_temp.copy()
        
        if (i % 100) == 0 or i == 1:
            print("Epoch: %s Batch: %s Error: %.8f lr: %.8f "%(it, i, error, eta))
        t += 1            
   
    return theta
def predict(theta, X):
    X = np.insert(X, values=1, obj=0, axis=1)
    return hyphotesis(theta_h, X)

max_iter = 10000
theta_h = SGD_(alpha, max_iter, X, y)
print (y,predict(theta_h, X))


In [ ]:
max_iter = 10000
theta_h = SGD_(1., max_iter=max_iter, X=X_train, y=np.log(y_train.values))


In [ ]:
y_pred = np.exp(predict(theta_h, X_val))

df = pd.DataFrame({'real': y_val, 'pred': y_pred})
ax = df.sort_values('real').plot.scatter('real', 'pred', figsize=(5, 5))
_ = ax.plot([y_val.min(), y_val.max()], [y_pred.min(), y_pred.max()], 'k--', lw=3)



In [ ]:
np.mean((np.log(y_pred) - np.log(y_val.values))**2)

In [ ]:
np.mean((y_pred - y_val.values)**2)

In [ ]:
print("MSE: %.3f" % metrics.mean_squared_error(y_val, y_pred))
print("MAE: %.3f" % metrics.mean_absolute_error(y_val, y_pred))
print('R2: %.3f' % metrics.r2_score(y_val, y_pred))

plt.hist(y_val, bins=100, color='blue', linewidth=3)
plt.show()
plt.hist(y_pred, bins=100, color='red', linewidth=3)
plt.show()